In [1]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import chi2_contingency, beta
from IPython.display import Image

# Data Collection

In [3]:
df = pd.read_csv('ab_data.csv')

In [4]:
df.sample(2)

,user_id,timestamp,group,landing_page,converted
205553,858699,2017-01-05 00:15:20.887122,control,old_page,0
153556,682788,2017-01-03 03:09:16.756344,treatment,new_page,0


In [8]:
start_time = datetime.datetime.strptime(df['timestamp'].min(), '%Y-%m-%d %H:%M:%S.%f')
end_time = datetime.datetime.strptime(df['timestamp'].max(), '%Y-%m-%d %H:%M:%S.%f')
data_duration = (end_time - start_time).days

print(f"Number of unique users in experiment:{df['user_id'].nunique()}")
print(f"Data collected for {data_duration} days")
print(f"Landing pages to compare : {df['landing_page'].unique().tolist()}")
print(f"Percentage of users in control : {round(df[df['group'] == 'control'].shape[0]*100/df.shape[0])}%")


Number of unique users in experiment:290584
Data collected for 21 days
Landing pages to compare : ['old_page', 'new_page']
Percentage of users in control : 50%


# Data Processing

there are some repeated exposures for some users

In [9]:
sample = df[df['user_id'].isin([746755,722274])]
sample

,user_id,timestamp,group,landing_page,converted
29073,746755,2017-01-11 01:28:57.083669,control,new_page,1
105487,722274,2017-01-19 01:46:53.093257,control,old_page,0
262554,722274,2017-01-09 21:21:23.638444,control,new_page,0
286566,746755,2017-01-05 03:40:08.457451,control,old_page,0


Get timestamp of first exposure

In [10]:
#1. get the timestamp of the first exposure
first_conversion = sample.groupby('user_id')['timestamp'].min().to_frame().reset_index()
sample = sample.merge(first_conversion, on=['user_id','timestamp'])

In [11]:
sample

,user_id,timestamp,group,landing_page,converted
0,722274,2017-01-09 21:21:23.638444,control,new_page,0
1,746755,2017-01-05 03:40:08.457451,control,old_page,0


See how many users have multiple buckets. If not many, remove them

In [13]:
counter = df['user_id'].value_counts()
(counter > 1).value_counts()

False    286690
True       3894
Name: user_id, dtype: int64

3894 (1.34%) user_ids have been exposed to the old AND new page. It should be okay to remove them

In [15]:
#Remove users with multiple buckets
valid_users = pd.DataFrame(counter[counter==1].index, columns=['user_id'])
df = df.merge(valid_users, on = ['user_id'])
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [17]:
# Add week column to see the data as you would during experiment
df['week'] = df['timestamp'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f').isocalendar()[1])
df.sample()


,user_id,timestamp,group,landing_page,converted,week
226160,742372,2017-01-02 23:17:12.358322,control,old_page,0,1


In [18]:
df['week'].value_counts()

2    91380
3    91056
1    83745
4    20509
Name: week, dtype: int64

# Experiment: Frequentist Approach

In [25]:
#Get Stats
NUM_WEEKS = 4  # Vary number to get experiment data at weekly points in time
experiment_data = df[df['week'] <= NUM_WEEKS]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round(treatment_conversion_perc - control_conversion_perc, 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.873%
Control Conversion Rate: 12.017%
Lift = -0.144%


**Chi-Squared Test**

H0: Control and Treatment are independent

HA: Control and Treatment are not independent


In [30]:
# Create Contingency Table for Chi Squared Test
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted
contingency_table = np.array([[control_converted, control_non_converted], 
                              [treatment_converted, treatment_non_converted]])
contingency_table

array([[ 17220, 126073],
       [ 17025, 126372]], dtype=int64)

In [28]:
chi, p_value, _, _ = chi2_contingency(contingency_table,correction = False)
chi, p_value

(1.426794609399621, 0.23228827305833816)

Since the p_value > 0.05, we cannot reject null hypothesis. Hence, we cannot conclude if there exists a relationship between the control and treatment groups

In [31]:
print(f"{round(p_value * 100, 2)}% probability that a more extreme chi square than {round(chi, 3)} would have occured by chance")


23.23% probability that a more extreme chi square than 1.427 would have occured by chance


But this is tough to interpret. We would to say something about the actual maginitude of lift. Something like this:

In [32]:
print(f"(We CANNOT say this) We are {round(p_value * 100, 2)}% confident that our lift = {lift}%")

(We CANNOT say this) We are 23.23% confident that our lift = -0.144%


# Experiment: Bayesian Approach

need to create prior distribution and have the experiment update the parameters to create posterier distributions

Since these prior & posterior distributions will be used to sample Conversion Rate, we model them after beta distribtion.

Let's create the prior beta distribtion from the first weeks of conversion data

In [33]:
prior = df[(df['week']==1) & (df['group']=='control')]

In [34]:
prior_means = []
for i in range(10000):
    prior_means.append(prior.sample(1000)['converted'].mean())
    

In [36]:
prior_means[:10]

[0.111, 0.135, 0.123, 0.124, 0.125, 0.121, 0.123, 0.122, 0.116, 0.131]

In [37]:
# Model Beta Distribution from sample means
prior_alpha, prior_beta, _, _ = beta.fit(prior_means, floc =0, fscale =1 )

In [38]:
# Get Stats
NUM_WEEKS = 4 # Vary number to get experiment data at weekly points in time
experiment_data = df[(df['week'] > 1) & (df['week'] <= NUM_WEEKS)]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round((treatment_conversion_perc - control_conversion_perc) / control_conversion_perc , 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.909%
Control Conversion Rate: 12.058%
Lift = -0.012%


In [39]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted

# Update Prior parameters with experiment conversion rates
posterior_control = beta(prior_alpha + control_converted, prior_beta + control_non_converted)
posterior_treatment = beta(prior_alpha + treatment_converted, prior_beta + treatment_non_converted)

# sample from Posteriors
control_samples = posterior_control.rvs(1000)
treatment_samples = posterior_treatment.rvs(1000)
probability = np.mean(treatment_samples > control_samples)
print(f"Probability that treatment > control: {probability * 100}%")


Probability that treatment > control: 14.7%


In [40]:
(control_mu),  (control_var) = posterior_control.stats()
(treatment_mu), (treatment_var) = posterior_treatment.stats()
print(f"Control Posterior: Mean: {control_mu}, Variance: {control_var}") 
print(f"Treatment Posterior: Mean: {treatment_mu}, Variance: {treatment_var}")

Control Posterior: Mean: 0.12056268339547102, Variance: 1.0337127473264945e-06
Treatment Posterior: Mean: 0.11909359341147184, Variance: 1.0246105411332793e-06


We can even make statements like the following which are actionable:

In [41]:
lift_percentage = (treatment_samples - control_samples) / control_samples
print(f"Probability that we are seeing a 2% lift: {np.mean((100 * lift_percentage) > 2) * 100}%")

Probability that we are seeing a 2% lift: 0.5%


In [51]:
# Get Stats
NUM_WEEKS = 2 # Vary number to get experiment data at weekly points in time
experiment_data = df[(df['week'] > 1) & (df['week'] <= NUM_WEEKS)]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round((treatment_conversion_perc - control_conversion_perc) / control_conversion_perc , 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.817%
Control Conversion Rate: 11.925%
Lift = -0.009%


In [52]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted

# Update Prior parameters with experiment conversion rates
posterior_control = beta(prior_alpha + control_converted, prior_beta + control_non_converted)
posterior_treatment = beta(prior_alpha + treatment_converted, prior_beta + treatment_non_converted)

# sample from Posteriors
control_samples = posterior_control.rvs(1000)
treatment_samples = posterior_treatment.rvs(1000)
probability = np.mean(treatment_samples > control_samples)
print(f"Probability that treatment > control: {probability * 100}%")

Probability that treatment > control: 30.0%


In [53]:
(control_mu),  (control_var) = posterior_control.stats()
(treatment_mu), (treatment_var) = posterior_treatment.stats()
print(f"Control Posterior: Mean: {control_mu}, Variance: {control_var}") 
print(f"Treatment Posterior: Mean: {treatment_mu}, Variance: {treatment_var}")

Control Posterior: Mean: 0.11924940287183315, Variance: 2.249379451837147e-06
Treatment Posterior: Mean: 0.11819606256462217, Variance: 2.231698930730576e-06


In [54]:
lift_percentage = (treatment_samples - control_samples) / control_samples
print(f"Probability that we are seeing a 2% lift: {np.mean((100 * lift_percentage) > 2) * 100}%")

Probability that we are seeing a 2% lift: 5.3%
